In [2]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import random

# Data preparation
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train)
X_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(y_test)

# Neural Network with configurable hidden layers
class FlexibleNN(nn.Module):
    def __init__(self, hidden_layers):
        super().__init__()
        self.layers = nn.ModuleList()

        # Input layer
        self.layers.append(nn.Linear(20, hidden_layers[0]))

        # Hidden layers
        for i in range(len(hidden_layers)-1):
            self.layers.append(nn.Linear(hidden_layers[i], hidden_layers[i+1]))

        # Output layer
        self.layers.append(nn.Linear(hidden_layers[-1], 1))

        self.activation = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        for i in range(len(self.layers)-1):
            x = self.activation(self.layers[i](x))
        x = self.sigmoid(self.layers[-1](x))
        return x

# Training function with backpropagation
def train_model(model, epochs=100):
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    accuracies = []
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train.reshape(-1, 1))
        loss.backward()  # Backpropagation
        optimizer.step()

        if epoch % 10 == 0:
            model.eval()
            with torch.no_grad():
                test_outputs = model(X_test)
                predictions = (test_outputs >= 0.5).float()
                accuracy = (predictions.reshape(-1) == y_test).float().mean()
                accuracies.append(accuracy.item())
                print(f'Hidden layers: {hidden_layers}, Epoch {epoch}, Accuracy: {accuracy:.4f}')

    return accuracies[-1]

# Genetic Algorithm for architecture optimization
class GeneticOptimizer:
    def __init__(self, pop_size=10, mutation_rate=0.2):
        self.pop_size = pop_size
        self.mutation_rate = mutation_rate

    def create_population(self):
        population = []
        for _ in range(self.pop_size):
            n_layers = random.randint(1, 4)
            hidden_layers = [random.randint(16, 128) for _ in range(n_layers)]
            population.append(hidden_layers)
        return population

    def mutate(self, architecture):
        if random.random() < self.mutation_rate:
            idx = random.randint(0, len(architecture)-1)
            architecture[idx] = random.randint(16, 128)
        return architecture

    def crossover(self, parent1, parent2):
        point = random.randint(1, min(len(parent1), len(parent2))-1)
        child = parent1[:point] + parent2[point:]
        return child

    def optimize(self, generations=5):
        population = self.create_population()
        best_architecture = None
        best_accuracy = 0

        for gen in range(generations):
            print(f"\nGeneration {gen+1}")

            # Evaluate current population
            fitness_scores = []
            for architecture in population:
                model = FlexibleNN(architecture)
                accuracy = train_model(model, epochs=50)
                fitness_scores.append(accuracy)

                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_architecture = architecture

            # Create new population
            new_population = []
            sorted_pop = [x for _, x in sorted(zip(fitness_scores, population), reverse=True)]

            # Keep best 2 architectures
            new_population.extend(sorted_pop[:2])

            # Create offspring
            while len(new_population) < self.pop_size:
                parent1 = random.choice(sorted_pop[:5])
                parent2 = random.choice(sorted_pop[:5])
                child = self.crossover(parent1, parent2)
                child = self.mutate(child)
                new_population.append(child)

            population = new_population

        return best_architecture, best_accuracy

# Demonstrate improvement with increasing hidden layers
hidden_layer_configs = [
    [32],
    [32, 16],
    [64, 32, 16],
    [128, 64, 32, 16]
]

print("Testing different hidden layer configurations:")
for hidden_layers in hidden_layer_configs:
    model = FlexibleNN(hidden_layers)
    accuracy = train_model(model)
    print(f"\nFinal accuracy with {len(hidden_layers)} hidden layers {hidden_layers}: {accuracy:.4f}")

# Apply genetic algorithm for optimization
print("\nOptimizing architecture using Genetic Algorithm:")
ga_optimizer = GeneticOptimizer()
best_architecture, best_accuracy = ga_optimizer.optimize()
print(f"\nBest architecture found: {best_architecture}")
print(f"Best accuracy achieved: {best_accuracy:.4f}")


Testing different hidden layer configurations:
Hidden layers: [32], Epoch 0, Accuracy: 0.4900
Hidden layers: [32], Epoch 10, Accuracy: 0.5600
Hidden layers: [32], Epoch 20, Accuracy: 0.6100
Hidden layers: [32], Epoch 30, Accuracy: 0.6600
Hidden layers: [32], Epoch 40, Accuracy: 0.7050
Hidden layers: [32], Epoch 50, Accuracy: 0.7200
Hidden layers: [32], Epoch 60, Accuracy: 0.7650
Hidden layers: [32], Epoch 70, Accuracy: 0.7750
Hidden layers: [32], Epoch 80, Accuracy: 0.7950
Hidden layers: [32], Epoch 90, Accuracy: 0.8100

Final accuracy with 1 hidden layers [32]: 0.8100
Hidden layers: [32, 16], Epoch 0, Accuracy: 0.4650
Hidden layers: [32, 16], Epoch 10, Accuracy: 0.4700
Hidden layers: [32, 16], Epoch 20, Accuracy: 0.5950
Hidden layers: [32, 16], Epoch 30, Accuracy: 0.6850
Hidden layers: [32, 16], Epoch 40, Accuracy: 0.7400
Hidden layers: [32, 16], Epoch 50, Accuracy: 0.7800
Hidden layers: [32, 16], Epoch 60, Accuracy: 0.8150
Hidden layers: [32, 16], Epoch 70, Accuracy: 0.8250
Hidden la